Ce notebook vise à traiter les données issues de ERA5 (au dessus du BF ou des mers). Toutes les variables seront mises en commun et sauvegardées dans deux fichiers (decadaires et mensuelles)

In [2]:
import os, sys
import pyproj
from pyproj import Proj 
from netCDF4 import Dataset
import netCDF4, h5netcdf, zarr
import xarray as xr
import shutil, glob
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime
import cftime
import subprocess
import shutil
from functools import reduce
from config import Config, Constant
import logging
from cdo import Cdo
import warnings
warnings.filterwarnings('ignore')

#### Concatenation des fichiers netcdf

In [3]:
def concat_nc_files(folder_name, output_filename="output.nc"):
    # Liste pour stocker les datasets
    datasets = []
    
    # Parcours des fichiers dans le dossier
    for filename in os.listdir(folder_name):
        if filename.endswith(".nc"):

            #change_missing_val(filename,folder_name)
            # Chargement du fichier NetCDF
            ds = xr.open_dataset(os.path.join(folder_name, filename), engine='netcdf4')
            datasets.append(ds) 

    # Concaténation des datasets
    combined = xr.concat(datasets, dim="time")
    
    # Sauvegarde dans un fichier NetCDF
    combined.to_netcdf(output_filename)

#### Changement des valeurs manquantes des fichiers netcdf

In [5]:
def change_missing_val(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(".nc"):
            full_input_path = os.path.join(input_folder, filename)
            full_output_path = os.path.join(output_folder, filename)
            
            ds = xr.open_dataset(full_input_path)
            ds = ds.where(~np.isnan(ds), other=np.nan)
            ds.to_netcdf(full_output_path)
            ds.close()

#### Fonction pour moyenne roulante

In [6]:
def calculate_rolling_average(df_data, num_months):
    df=df_data.copy()
    # Triez le DataFrame par 'Station', 'Year' et 'Month'
    df = df.sort_values(['Station', 'Year', 'Month'])
    columns_to_average=['v_wind_925','u_wind_850', 'u_wind_700', 'u_wind_200', 'eau_precipitable', 't_point_rosee', 'h_vol_sol_wat', 'anom_lef_mois', 'anom_nino_mois']
    # Utilisez la méthode rolling() pour calculer la moyenne roulante sur le nombre de mois spécifié
    rolling_average = df.groupby('Station')[columns_to_average].rolling(window=num_months).mean()
    # Réinitialisez l'index
    rolling_average = rolling_average.reset_index()
    rolling_average = rolling_average.set_index('level_1')
    for col in columns_to_average:
        df[col] = rolling_average[col]
    return df

#### Gestion des colonnes date

In [7]:
def manage_date(all_data):
    df=all_data.copy()
    df['Date-Time'] = pd.to_datetime(df['Date-Time'])
    # Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
    decade_ranges = [0, 10, 20, 32]
    dek_num = [1, 2, 3]
    df['Day']=df['Date-Time'].dt.day
    df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
    # Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
    df['Year'] = df['Date-Time'].dt.year
    df['Month'] = df['Date-Time'].dt.month
    return df

##### Calcul des moyennes decadaires à partir de données trihoraires

In [8]:
def calcul_moy_decadaire(df_data, variable):
    df=df_data.copy()
    # Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
    decade_avg = df.groupby(['Station','Year', 'Month', 'Decade'])[variable].mean()
    # Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
    decade_avg = decade_avg.reset_index()
    #Ajout du nom de la dataframe pour la concatenation
    df_terre_dek.append(decade_avg)
    # Affichez les moyennes décadaires
    print("Moyenne décadaire :\n", decade_avg)

##### Calcul des moyennes mensuelles à partir de données trihoraires

In [9]:
def calcul_moy_mensuelle(df_data,variable):
    df=df_data.copy()
    # Maintenant, calculez la moyenne mensuelle en utilisant le même processus
    month_avg = df.groupby(['Station','Year', 'Month'])[variable].mean()
    month_avg = month_avg.reset_index()
    # Affichez les moyennes mensuelles
    print("\nMoyenne mensuelle :\n", month_avg)
    df_terre_mon.append(month_avg)

In [10]:
#Lecture du fichier des données des coordonnées des stations
#Ce fichier permet d'extraire les valeurs au dessus des stations à partir des fichiers NetCDF
station_infos_file=os.path.join(Config.STATION_PATH,Config.STAT_COORD_FILE_NAME)

stations = pd.read_excel(station_infos_file)

In [11]:
# Liste contenant les noms des dataframes decades et mensuels
df_terre_dek=[]
df_terre_mon=[]
df_mer_dek=[]
df_mer_mon=[]

### I- VITESSE VENT V À LA 925 hPa

##### Changement des données manquantes du fichiers decadaires netcdf

In [12]:
# Variables de configuration
folder_original = Config.V_WIND_925_DEK
wind_925_v = Config.WIND_V_925
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.V_WIND_925_VAL
change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [13]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, wind_925_v)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [14]:
# Lecture de fichier de vent v à la 925hPa
v_wind_925=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
v_wind=v_wind_925.variables['v'][:]
unitd=v_wind_925.variables['v'].units
unit=v_wind_925.variables['time'].units
times=v_wind_925.variables['time'][:]
lat=v_wind_925.variables['latitude'][:]
lon=v_wind_925.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [15]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    v_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        v_wind_data.append(v_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'v_wind_925': v_wind_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_V_925_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\v_wind_925.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\v_wind_925.csv


In [16]:
all_data.head()

,Station,Date-Time,v_wind_925
0,Bobo_Dioulasso,1961-01-01 00:00:00,-6.685907
1,Bobo_Dioulasso,1961-01-01 03:00:00,-6.219835
2,Bobo_Dioulasso,1961-01-01 06:00:00,-5.208807
3,Bobo_Dioulasso,1961-01-01 09:00:00,-4.032507
4,Bobo_Dioulasso,1961-01-01 12:00:00,-2.147271


In [17]:
df=manage_date(all_data)

##### I-1 CALCUL DES MOYENNES DECADAIRES 

In [18]:
calcul_moy_decadaire(df, 'v_wind_925')

Moyenne décadaire :
               Station  Year  Month Decade  v_wind_925
0      Bobo_Dioulasso  1961      1      1   -5.685185
1      Bobo_Dioulasso  1961      1      2   -5.627637
2      Bobo_Dioulasso  1961      1      3   -7.595885
3      Bobo_Dioulasso  1961      2      1   -5.607470
4      Bobo_Dioulasso  1961      2      2   -5.117176
...               ...   ...    ...    ...         ...
22675              Po  2023     11      2   -0.520774
22676              Po  2023     11      3    1.036428
22677              Po  2023     12      1   -2.256523
22678              Po  2023     12      2   -5.118263
22679              Po  2023     12      3   -3.944526

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  v_wind_925
0      Bobo_Dioulasso  1961      1      1   -5.685185
1      Bobo_Dioulasso  1961      1      2   -5.627637
2      Bobo_Dioulasso  1961      1      3   -7.595885
3      Bobo_Dioulasso  1961      2      1   -5.607470
4      Bobo_Di

##### I-2 CALCUL DES MOYENNES MENSUELLES

In [19]:
calcul_moy_mensuelle(df,'v_wind_925')


Moyenne mensuelle :
              Station  Year  Month  v_wind_925
0     Bobo_Dioulasso  1961      1   -6.344612
1     Bobo_Dioulasso  1961      2   -4.787571
2     Bobo_Dioulasso  1961      3   -2.121944
3     Bobo_Dioulasso  1961      4    4.142627
4     Bobo_Dioulasso  1961      5    3.927357
...              ...   ...    ...         ...
7555              Po  2023      8    2.812753
7556              Po  2023      9    2.504891
7557              Po  2023     10    2.327498
7558              Po  2023     11    0.983150
7559              Po  2023     12   -3.778634

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  v_wind_925
0     Bobo_Dioulasso  1961      1   -6.344612
1     Bobo_Dioulasso  1961      2   -4.787571
2     Bobo_Dioulasso  1961      3   -2.121944
3     Bobo_Dioulasso  1961      4    4.142627
4     Bobo_Dioulasso  1961      5    3.927357
...              ...   ...    ...         ...
7555              Po  2023      8    2.812753
7556      

### II- COMPOSANTE U DU VENT À LA 850 hPa

##### Changement des données manquantes du fichiers decadaires netcdf

In [20]:
# Variables de configuration
folder_original = Config.U_WIND_850_DEK
wind_850_U = Config.WIND_U_850
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.U_WIND_850_VAL
# Chemin de sortie
change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [21]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, wind_850_U)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [22]:
# Lecture de fichier de vent u à la 850hPa
u_wind_850=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
u_wind=u_wind_850.variables['u'][:]
unitd=u_wind_850.variables['u'].units
unit=u_wind_850.variables['time'].units
times=u_wind_850.variables['time'][:]
lat=u_wind_850.variables['latitude'][:]
lon=u_wind_850.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [23]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    u_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        u_wind_data.append(u_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'u_wind_850': u_wind_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_U_850_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_850.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_850.csv


In [24]:
all_data.head()

,Station,Date-Time,u_wind_850
0,Bobo_Dioulasso,1961-01-01 00:00:00,-3.469058
1,Bobo_Dioulasso,1961-01-01 03:00:00,-2.635350
2,Bobo_Dioulasso,1961-01-01 06:00:00,-1.584786
3,Bobo_Dioulasso,1961-01-01 09:00:00,-1.917576
4,Bobo_Dioulasso,1961-01-01 12:00:00,-1.623354


In [25]:
df=manage_date(all_data)

##### II-1 CALCUL DES MOYENNES DECADAIRES 

In [26]:
calcul_moy_decadaire(df, 'u_wind_850')

Moyenne décadaire :
               Station  Year  Month Decade  u_wind_850
0      Bobo_Dioulasso  1961      1      1   -4.901044
1      Bobo_Dioulasso  1961      1      2   -1.549461
2      Bobo_Dioulasso  1961      1      3   -0.394695
3      Bobo_Dioulasso  1961      2      1   -3.607335
4      Bobo_Dioulasso  1961      2      2   -3.535849
...               ...   ...    ...    ...         ...
22675              Po  2023     11      2   -6.118764
22676              Po  2023     11      3   -1.067482
22677              Po  2023     12      1   -5.201837
22678              Po  2023     12      2   -3.965210
22679              Po  2023     12      3   -2.022182

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  u_wind_850
0      Bobo_Dioulasso  1961      1      1   -4.901044
1      Bobo_Dioulasso  1961      1      2   -1.549461
2      Bobo_Dioulasso  1961      1      3   -0.394695
3      Bobo_Dioulasso  1961      2      1   -3.607335
4      Bobo_Di

##### II-2 CALCUL DES MOYENNES MENSUELLES

In [27]:
calcul_moy_mensuelle(df,'u_wind_850')


Moyenne mensuelle :
              Station  Year  Month  u_wind_850
0     Bobo_Dioulasso  1961      1   -2.220861
1     Bobo_Dioulasso  1961      2   -4.507234
2     Bobo_Dioulasso  1961      3   -3.204388
3     Bobo_Dioulasso  1961      4   -1.348264
4     Bobo_Dioulasso  1961      5   -0.323172
...              ...   ...    ...         ...
7555              Po  2023      8    0.693429
7556              Po  2023      9   -2.076714
7557              Po  2023     10   -4.663396
7558              Po  2023     11   -3.242003
7559              Po  2023     12   -3.674660

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  u_wind_850
0     Bobo_Dioulasso  1961      1   -2.220861
1     Bobo_Dioulasso  1961      2   -4.507234
2     Bobo_Dioulasso  1961      3   -3.204388
3     Bobo_Dioulasso  1961      4   -1.348264
4     Bobo_Dioulasso  1961      5   -0.323172
...              ...   ...    ...         ...
7555              Po  2023      8    0.693429
7556      

### III- COMPOSANTE U DU VENT À LA 700 hPa

##### Changement des données manquantes du fichiers decadaires netcdf

In [28]:
# Variables de configuration
folder_original = Config.U_WIND_700_DEK
wind_700_u = Config.WIND_U_700
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.U_WIND_700_VAL

change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [29]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, wind_700_u)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [30]:
# Lecture de fichier de vent v à la 700hPa
u_wind_700=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
u_wind=u_wind_700.variables['u'][:]
unitd=u_wind_700.variables['u'].units
unit=u_wind_700.variables['time'].units
times=u_wind_700.variables['time'][:]
lat=u_wind_700.variables['latitude'][:]
lon=u_wind_700.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [31]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    u_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        u_wind_data.append(u_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'u_wind_700': u_wind_data})
    

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_U_700_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_700.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_700.csv


In [32]:
all_data.head()

,Station,Date-Time,u_wind_700
0,Bobo_Dioulasso,1961-01-01 00:00:00,-3.787606
1,Bobo_Dioulasso,1961-01-01 03:00:00,-3.507797
2,Bobo_Dioulasso,1961-01-01 06:00:00,-4.108529
3,Bobo_Dioulasso,1961-01-01 09:00:00,-5.171272
4,Bobo_Dioulasso,1961-01-01 12:00:00,-3.107604


In [33]:
df=manage_date(all_data)

##### III-1 CALCUL DES MOYENNES DECADAIRES 

In [34]:
calcul_moy_decadaire(df, 'u_wind_700')

Moyenne décadaire :
               Station  Year  Month Decade  u_wind_700
0      Bobo_Dioulasso  1961      1      1   -6.507310
1      Bobo_Dioulasso  1961      1      2   -1.188504
2      Bobo_Dioulasso  1961      1      3    2.207007
3      Bobo_Dioulasso  1961      2      1    3.229098
4      Bobo_Dioulasso  1961      2      2    3.110321
...               ...   ...    ...    ...         ...
22675              Po  2023     11      2   -6.794710
22676              Po  2023     11      3   -3.191192
22677              Po  2023     12      1   -4.503232
22678              Po  2023     12      2    0.133786
22679              Po  2023     12      3   -4.684278

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  u_wind_700
0      Bobo_Dioulasso  1961      1      1   -6.507310
1      Bobo_Dioulasso  1961      1      2   -1.188504
2      Bobo_Dioulasso  1961      1      3    2.207007
3      Bobo_Dioulasso  1961      2      1    3.229098
4      Bobo_Di

##### III-2 CALCUL DES MOYENNES MENSUELLES

In [35]:
calcul_moy_mensuelle(df,'u_wind_700')


Moyenne mensuelle :
              Station  Year  Month  u_wind_700
0     Bobo_Dioulasso  1961      1   -1.699389
1     Bobo_Dioulasso  1961      2    0.534085
2     Bobo_Dioulasso  1961      3   -0.900405
3     Bobo_Dioulasso  1961      4   -4.686925
4     Bobo_Dioulasso  1961      5   -8.638417
...              ...   ...    ...         ...
7555              Po  2023      8   -6.635411
7556              Po  2023      9  -11.216813
7557              Po  2023     10  -10.574696
7558              Po  2023     11   -4.986422
7559              Po  2023     12   -3.071662

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  u_wind_700
0     Bobo_Dioulasso  1961      1   -1.699389
1     Bobo_Dioulasso  1961      2    0.534085
2     Bobo_Dioulasso  1961      3   -0.900405
3     Bobo_Dioulasso  1961      4   -4.686925
4     Bobo_Dioulasso  1961      5   -8.638417
...              ...   ...    ...         ...
7555              Po  2023      8   -6.635411
7556      

### IV- COMPOSANTE U DU VENT À LA 200 hPa

##### Changement des données manquantes du fichiers decadaires netcdf

In [36]:
# Variables de configuration
folder_original = Config.U_WIND_200_DEK
wind_200_u = Config.WIND_U_200
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.U_WIND_200_VAL

change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [37]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, wind_200_u)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [38]:
# Lecture de fichier de vent v à la 200hPa
u_wind_200=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
u_wind=u_wind_200.variables['u'][:]
unitd=u_wind_200.variables['u'].units
unit=u_wind_200.variables['time'].units
times=u_wind_200.variables['time'][:]
lat=u_wind_200.variables['latitude'][:]
lon=u_wind_200.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [39]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    u_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        u_wind_data.append(u_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'u_wind_200': u_wind_data})
    

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_U_200_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_200.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\u_wind_200.csv


In [40]:
all_data.head()

,Station,Date-Time,u_wind_200
0,Bobo_Dioulasso,1961-01-01 00:00:00,20.978663
1,Bobo_Dioulasso,1961-01-01 03:00:00,19.108212
2,Bobo_Dioulasso,1961-01-01 06:00:00,19.835911
3,Bobo_Dioulasso,1961-01-01 09:00:00,20.103376
4,Bobo_Dioulasso,1961-01-01 12:00:00,19.063032


In [41]:
df=manage_date(all_data)

##### IV-1 CALCUL DES MOYENNES DECADAIRES 

In [42]:
calcul_moy_decadaire(df, 'u_wind_200')

Moyenne décadaire :
               Station  Year  Month Decade  u_wind_200
0      Bobo_Dioulasso  1961      1      1   24.359285
1      Bobo_Dioulasso  1961      1      2   28.219953
2      Bobo_Dioulasso  1961      1      3   32.303318
3      Bobo_Dioulasso  1961      2      1   12.524779
4      Bobo_Dioulasso  1961      2      2   19.788904
...               ...   ...    ...    ...         ...
22675              Po  2023     11      2   13.294998
22676              Po  2023     11      3    3.461331
22677              Po  2023     12      1    4.892241
22678              Po  2023     12      2   16.447259
22679              Po  2023     12      3   20.191202

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  u_wind_200
0      Bobo_Dioulasso  1961      1      1   24.359285
1      Bobo_Dioulasso  1961      1      2   28.219953
2      Bobo_Dioulasso  1961      1      3   32.303318
3      Bobo_Dioulasso  1961      2      1   12.524779
4      Bobo_Di

##### IV-2 CALCUL DES MOYENNES MENSUELLES

In [43]:
calcul_moy_mensuelle(df,'u_wind_200')


Moyenne mensuelle :
              Station  Year  Month  u_wind_200
0     Bobo_Dioulasso  1961      1   28.423512
1     Bobo_Dioulasso  1961      2   17.062947
2     Bobo_Dioulasso  1961      3   22.134290
3     Bobo_Dioulasso  1961      4   13.424217
4     Bobo_Dioulasso  1961      5    5.019922
...              ...   ...    ...         ...
7555              Po  2023      8  -11.575755
7556              Po  2023      9   -7.026378
7557              Po  2023     10    3.732810
7558              Po  2023     11    6.012696
7559              Po  2023     12   14.048330

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  u_wind_200
0     Bobo_Dioulasso  1961      1   28.423512
1     Bobo_Dioulasso  1961      2   17.062947
2     Bobo_Dioulasso  1961      3   22.134290
3     Bobo_Dioulasso  1961      4   13.424217
4     Bobo_Dioulasso  1961      5    5.019922
...              ...   ...    ...         ...
7555              Po  2023      8  -11.575755
7556      

### V- EAU PRECIPITABLE

##### Changement des données manquantes du fichiers decadaires netcdf

In [44]:
# Variables de configuration
folder_original = Config.PREC_EAU_DEK
eau_precip = Config.EAU_PREC
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.PREC_EAU_VAL

change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [45]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, eau_precip)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [46]:
# Lecture de fichier de vent u à la 700 hPa
eau_precipitable=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
eau_prec=eau_precipitable.variables['p79.162'][:]
unitd=eau_precipitable.variables['p79.162'].units
unit=eau_precipitable.variables['time'].units
times=eau_precipitable.variables['time'][:]
lat=eau_precipitable.variables['latitude'][:]
lon=eau_precipitable.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [47]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    eau_prec_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        eau_prec_data.append(eau_prec[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  #expv[1] Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'eau_precipitable': eau_prec_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.EAU_PREC_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")

Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\eau_prec.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\eau_prec.csv


In [48]:
all_data.head()

,Station,Date-Time,eau_precipitable
0,Bobo_Dioulasso,1961-01-01 00:00:00,-3.441911e-07
1,Bobo_Dioulasso,1961-01-01 03:00:00,-1.278351e-06
2,Bobo_Dioulasso,1961-01-01 06:00:00,-1.777733e-06
3,Bobo_Dioulasso,1961-01-01 09:00:00,-3.109544e-07
4,Bobo_Dioulasso,1961-01-01 12:00:00,-2.214230e-07


In [49]:
df=manage_date(all_data)

##### V-1 CALCUL DES MOYENNES DECADAIRES 

In [50]:
calcul_moy_decadaire(df, 'eau_precipitable')

Moyenne décadaire :
               Station  Year  Month Decade  eau_precipitable
0      Bobo_Dioulasso  1961      1      1      5.872143e-09
1      Bobo_Dioulasso  1961      1      2      1.950160e-08
2      Bobo_Dioulasso  1961      1      3     -7.126785e-07
3      Bobo_Dioulasso  1961      2      1     -3.006926e-08
4      Bobo_Dioulasso  1961      2      2     -4.638627e-08
...               ...   ...    ...    ...               ...
22675              Po  2023     11      2      1.995109e-07
22676              Po  2023     11      3      6.656191e-07
22677              Po  2023     12      1     -6.186982e-09
22678              Po  2023     12      2     -1.006486e-07
22679              Po  2023     12      3      2.643045e-09

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  eau_precipitable
0      Bobo_Dioulasso  1961      1      1      5.872143e-09
1      Bobo_Dioulasso  1961      1      2      1.950160e-08
2      Bobo_Dioulasso  1961     

##### V-2 CALCUL DES MOYENNES MENSUELLES

In [51]:
calcul_moy_mensuelle(df,'eau_precipitable')


Moyenne mensuelle :
              Station  Year  Month  eau_precipitable
0     Bobo_Dioulasso  1961      1     -2.447008e-07
1     Bobo_Dioulasso  1961      2     -4.154257e-08
2     Bobo_Dioulasso  1961      3      1.196773e-07
3     Bobo_Dioulasso  1961      4      1.468938e-08
4     Bobo_Dioulasso  1961      5     -6.044959e-08
...              ...   ...    ...               ...
7555              Po  2023      8     -4.339784e-07
7556              Po  2023      9      3.548711e-07
7557              Po  2023     10      1.275398e-07
7558              Po  2023     11      4.417822e-07
7559              Po  2023     12     -3.352524e-08

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  eau_precipitable
0     Bobo_Dioulasso  1961      1     -2.447008e-07
1     Bobo_Dioulasso  1961      2     -4.154257e-08
2     Bobo_Dioulasso  1961      3      1.196773e-07
3     Bobo_Dioulasso  1961      4      1.468938e-08
4     Bobo_Dioulasso  1961      5     -6.04495

### V- TEMPERATURE DE POINT DE ROSÉE

##### Changement des données manquantes du fichiers decadaires netcdf

In [52]:
# Variables de configuration
folder_original = Config.TEMP_ROS_DEK
point_rosee = Config.POINT_ROSEE
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.TEMP_ROS_VAL

change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [53]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, point_rosee)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [54]:
# Lecture de fichier de vent u à la 700 hPa
t_point_rosee=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
point_rosee=t_point_rosee.variables['d2m'][:]
unitd=t_point_rosee.variables['d2m'].units
unit=t_point_rosee.variables['time'].units
times=t_point_rosee.variables['time'][:]
lat=t_point_rosee.variables['latitude'][:]
lon=t_point_rosee.variables['longitude'][:]


##### Recuperation des valeurs les plus proche de chaque station

In [55]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    point_rosee_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        point_rosee_data.append(point_rosee[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station,'Date-Time': date_range, 't_point_rosee': point_rosee_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.POINT_ROSEE_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\point_rosee.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\point_rosee.csv


In [56]:
all_data.head()

,Station,Date-Time,t_point_rosee
0,Bobo_Dioulasso,1961-01-01 00:00:00,270.459699
1,Bobo_Dioulasso,1961-01-01 03:00:00,270.892175
2,Bobo_Dioulasso,1961-01-01 06:00:00,270.785854
3,Bobo_Dioulasso,1961-01-01 09:00:00,269.847453
4,Bobo_Dioulasso,1961-01-01 12:00:00,271.654914


In [57]:
df=manage_date(all_data)

##### V-1 CALCUL DES MOYENNES DECADAIRES 

In [58]:
calcul_moy_decadaire(df, 't_point_rosee')

Moyenne décadaire :
               Station  Year  Month Decade  t_point_rosee
0      Bobo_Dioulasso  1961      1      1     274.505384
1      Bobo_Dioulasso  1961      1      2     274.479259
2      Bobo_Dioulasso  1961      1      3     273.833101
3      Bobo_Dioulasso  1961      2      1     269.842157
4      Bobo_Dioulasso  1961      2      2     269.390330
...               ...   ...    ...    ...            ...
22675              Po  2023     11      2     288.788924
22676              Po  2023     11      3     291.597863
22677              Po  2023     12      1     283.625472
22678              Po  2023     12      2     278.778910
22679              Po  2023     12      3     277.919467

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  t_point_rosee
0      Bobo_Dioulasso  1961      1      1     274.505384
1      Bobo_Dioulasso  1961      1      2     274.479259
2      Bobo_Dioulasso  1961      1      3     273.833101
3      Bobo_Dioulass

##### V-2 CALCUL DES MOYENNES MENSUELLES

In [59]:
calcul_moy_mensuelle(df,'t_point_rosee')


Moyenne mensuelle :
              Station  Year  Month  t_point_rosee
0     Bobo_Dioulasso  1961      1     274.258404
1     Bobo_Dioulasso  1961      2     270.738747
2     Bobo_Dioulasso  1961      3     275.947227
3     Bobo_Dioulasso  1961      4     291.274081
4     Bobo_Dioulasso  1961      5     294.604530
...              ...   ...    ...            ...
7555              Po  2023      8     297.007865
7556              Po  2023      9     297.335975
7557              Po  2023     10     296.495194
7558              Po  2023     11     291.622302
7559              Po  2023     12     280.037353

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  t_point_rosee
0     Bobo_Dioulasso  1961      1     274.258404
1     Bobo_Dioulasso  1961      2     270.738747
2     Bobo_Dioulasso  1961      3     275.947227
3     Bobo_Dioulasso  1961      4     291.274081
4     Bobo_Dioulasso  1961      5     294.604530
...              ...   ...    ...            ...

### VI- HUMIDITE DU SOL

##### Changement des données manquantes du fichiers decadaires netcdf

In [60]:
# Variables de configuration
folder_original = Config.SOL_EAU_DEK
sol_eau = Config.SOL_WATERPOINT
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.SOL_EAU_VAL

change_missing_val(folder_original,folder_miss_check)

##### Concatenation des fichiers decadaires

In [61]:
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, sol_eau)
concat_nc_files(folder_miss_check, output_path)

##### Lecture du fichier concatené

In [62]:
# Lecture de fichier de vent u à la 700 hPa
h_vol_sol_wat=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
volum_wat=h_vol_sol_wat.variables['swvl1'][:]
unitd=h_vol_sol_wat.variables['swvl1'].units
unit=h_vol_sol_wat.variables['time'].units
times=h_vol_sol_wat.variables['time'][:]
lat=h_vol_sol_wat.variables['latitude'][:]
lon=h_vol_sol_wat.variables['longitude'][:]

##### Recuperation des valeurs les plus proche de chaque station

In [63]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    volum_wat_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        volum_wat_data.append(volum_wat[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station,'Date-Time': date_range, 'h_vol_sol_wat': volum_wat_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.SOL_WATERPOINT_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")



Generating time series for Bobo_Dioulasso
Generating time series for Bobo_Dioulasso


Generating time series for Bogande
Generating time series for Bogande


Generating time series for Boromo
Generating time series for Boromo


Generating time series for Dedougou
Generating time series for Dedougou


Generating time series for Dori
Generating time series for Dori


...

Generating time series for Gaoua
Generating time series for Gaoua


Generating time series for Ouagadougou_aero
Generating time series for Ouagadougou_aero


Generating time series for Ouahigouya
Generating time series for Ouahigouya


Generating time series for Po
Generating time series for Po


All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\vol_soil_water.csv
All station data is saved in the CSV file: ..\ASSETS\DATA\ERA5\vol_soil_water.csv


In [64]:
all_data.head()

,Station,Date-Time,h_vol_sol_wat
0,Bobo_Dioulasso,1961-01-01 00:00:00,0.167686
1,Bobo_Dioulasso,1961-01-01 03:00:00,0.167662
2,Bobo_Dioulasso,1961-01-01 06:00:00,0.167653
3,Bobo_Dioulasso,1961-01-01 09:00:00,0.167629
4,Bobo_Dioulasso,1961-01-01 12:00:00,0.167174


In [65]:
df=manage_date(all_data)

##### V-1 CALCUL DES MOYENNES DECADAIRES 

In [66]:
calcul_moy_decadaire(df, 'h_vol_sol_wat')

Moyenne décadaire :
               Station  Year  Month Decade  h_vol_sol_wat
0      Bobo_Dioulasso  1961      1      1       0.177588
1      Bobo_Dioulasso  1961      1      2       0.178358
2      Bobo_Dioulasso  1961      1      3       0.178193
3      Bobo_Dioulasso  1961      2      1       0.172408
4      Bobo_Dioulasso  1961      2      2       0.173361
...               ...   ...    ...    ...            ...
22675              Po  2023     11      2       0.125040
22676              Po  2023     11      3       0.120388
22677              Po  2023     12      1       0.122402
22678              Po  2023     12      2       0.122092
22679              Po  2023     12      3       0.123349

[22680 rows x 5 columns]
Moyenne décadaire :
               Station  Year  Month Decade  h_vol_sol_wat
0      Bobo_Dioulasso  1961      1      1       0.177588
1      Bobo_Dioulasso  1961      1      2       0.178358
2      Bobo_Dioulasso  1961      1      3       0.178193
3      Bobo_Dioulass

##### V-2 CALCUL DES MOYENNES MENSUELLES

In [67]:
calcul_moy_mensuelle(df,'h_vol_sol_wat')


Moyenne mensuelle :
              Station  Year  Month  h_vol_sol_wat
0     Bobo_Dioulasso  1961      1       0.178051
1     Bobo_Dioulasso  1961      2       0.173297
2     Bobo_Dioulasso  1961      3       0.170007
3     Bobo_Dioulasso  1961      4       0.204349
4     Bobo_Dioulasso  1961      5       0.262240
...              ...   ...    ...            ...
7555              Po  2023      8       0.378430
7556              Po  2023      9       0.375144
7557              Po  2023     10       0.227509
7558              Po  2023     11       0.123016
7559              Po  2023     12       0.122638

[7560 rows x 4 columns]


Moyenne mensuelle :
              Station  Year  Month  h_vol_sol_wat
0     Bobo_Dioulasso  1961      1       0.178051
1     Bobo_Dioulasso  1961      2       0.173297
2     Bobo_Dioulasso  1961      3       0.170007
3     Bobo_Dioulasso  1961      4       0.204349
4     Bobo_Dioulasso  1961      5       0.262240
...              ...   ...    ...            ...

### VII- PROCESSING LANGUE D'EAU FROIDE DATA

##### Changement des données manquantes du fichiers decadaires netcdf

In [68]:
# Variables de configuration
folder_original = Config.SST_LEF_DEK
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.SST_LEF_VAL
change_missing_val(folder_original,folder_miss_check)

##### Calcul de la moyenne spatiale

In [70]:
# Dossier d'entrée contenant les fichiers SST
input_directory = Config.SST_LEF_VAL

# Dossier de sortie pour les fichiers résultants
output_directory = Config.SST_LEF_SP_MN_PATH

# Vérifier les chemins des dossiers
print(f"Dossier d'entrée : {input_directory}")
print(f"Dossier de sortie : {output_directory}")

# Si le dossier de sortie existe, supprimez-le
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

# Recréez le dossier de sortie
os.makedirs(output_directory)

# Boucle sur les fichiers dans le dossier d'entrée
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        # Chemin complet vers le fichier d'entrée
        input_file = os.path.join(input_directory, filename)
        
        # Déterminez le nom du fichier de sortie en ajoutant "_fldmean" au nom d'origine
        output_filename = os.path.splitext(filename)[0] + '_fldmean.nc'
        output_file = os.path.join(output_directory, output_filename)
        
        # Ouvrir le fichier NetCDF avec xarray
        ds = xr.open_dataset(input_file)
        
        # Calculer la moyenne spatiale
        ds_mean = ds.mean(dim=['latitude', 'longitude'])
        
        # Sauvegarder le résultat dans un nouveau fichier NetCDF
        ds_mean.to_netcdf(output_file)
        
        #print(f"Fichier généré : {output_file}")

print("Calcul de la moyenne spatiale (fldmean) terminé.")


Dossier d'entrée : ..\ASSETS\DATA\RAW\ERA5\SST_LEF_VAL
Dossier de sortie : ..\ASSETS\DATA\RAW\ERA5\SST_LEF_FLDMEAN
Dossier d'entrée : ..\ASSETS\DATA\RAW\ERA5\SST_LEF_VAL
Dossier de sortie : ..\ASSETS\DATA\RAW\ERA5\SST_LEF_FLDMEAN


Calcul de la moyenne spatiale (fldmean) terminé.
Calcul de la moyenne spatiale (fldmean) terminé.


##### Concatenation des fichiers après moyenne spatiale

In [71]:
dir_out_era5 = Config.ERA5_PATH
output_merged_file = os.path.join(dir_out_era5, Config.SST_LEF_MERGE)
to_concat_directory = Config.SST_LEF_SP_MN_PATH

# Liste de fichiers de sortie à concaténer
output_files_to_concatenate = [os.path.join(to_concat_directory, filename) for filename in os.listdir(to_concat_directory) if filename.endswith('_fldmean.nc')]

datasets = [xr.open_dataset(f) for f in output_files_to_concatenate]

# Concaténez les datasets le long de la dimension temporelle
combined = xr.concat(datasets, dim='time')

# Construisez le chemin complet pour le fichier de sortie
output_merged_file= os.path.join(dir_out_era5, Config.SST_LEF_MERGE)

# Enregistrez le dataset combiné dans un fichier
combined.to_netcdf(output_merged_file)

print(f"Le fichier combiné a été enregistré à {output_merged_file}")


Le fichier combiné a été enregistré à ..\ASSETS\DATA\RAW\ERA5\sst_leg_merged.nc
Le fichier combiné a été enregistré à ..\ASSETS\DATA\RAW\ERA5\sst_leg_merged.nc


##### Transformation du fichier netcdf en dataframe

In [72]:
# Ouvrir le fichier NetCDF
data = netCDF4.Dataset(output_merged_file)
# Extraire les variables time et sst
time_values = data.variables['time'][:]
sst_values = data.variables['sst'][:]

# Redimensionner les tableaux pour les rendre 1D
time_values = time_values.flatten()
sst_values = sst_values.flatten()
# Extraire les attributs de la variable "time"
time_var = data.variables['time']
units = time_var.units  # Obtenez l'unité de temps
# Fermer le fichier NetCDF
data.close()

# Convertir les valeurs de "time" en dates en utilisant cftime
dates = cftime.num2date(time_values, units, calendar='standard')

# Créer un DataFrame Pandas avec les dates converties
df_merged_sst_lef = pd.DataFrame({'date': dates, 'sst': sst_values})

In [73]:
# Utilisez apply pour convertir chaque valeur en datetime
df_merged_sst_lef['date'] = df_merged_sst_lef['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if isinstance(x, cftime._cftime.DatetimeGregorian) else x)
df_merged_sst_lef['date'] = pd.to_datetime(df_merged_sst_lef['date'])

In [74]:
df_merged_sst_lef['Year'] = df_merged_sst_lef['date'].dt.year
# Créez la colonne 'Mois' en extrayant le mois
df_merged_sst_lef['Month'] = df_merged_sst_lef['date'].dt.month
# Créez la colonne 'Jour' en extrayant le jour
df_merged_sst_lef['Day'] = df_merged_sst_lef['date'].dt.day
df_merged_sst_lef['Decade']=pd.cut(df_merged_sst_lef['Day'],bins=[0,10,20,32], labels=[1,2,3])

#### VII.1 - CALCUL DECADAIRE

In [75]:
df_sst_lef_dek=df_merged_sst_lef.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_lef_dek['Moy_lef_Dec'] = df_sst_lef_dek.groupby(['Year', 'Month', 'Decade'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_lef_dek.drop_duplicates(subset=['Year', 'Month', 'Decade', 'Moy_lef_Dec'], inplace=True)
# Affichez le DataFrame résultant
df_sst_lef_dek.head()

,date,sst,Year,Month,Day,Decade,Moy_lef_Dec
0,1961-01-01,300.468854,1961,1,1,1,300.383950
80,1961-01-11,300.123720,1961,1,11,2,300.250248
160,1961-01-21,300.506214,1961,1,21,3,300.686872
248,1961-02-01,300.968978,1961,2,1,1,300.884871
328,1961-02-11,300.888531,1961,2,11,2,301.116671


##### Calcul anomalies

In [76]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_lef_dek[(df_sst_lef_dek['Year'] >= ref_start) & (df_sst_lef_dek['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_lef_dek['anom_lef_dek']=df_sst_lef_dek['Moy_lef_Dec']-sst_mean
df_sst_lef_dek=df_sst_lef_dek[['Year','Month','Decade','anom_lef_dek']]
df_mer_dek.append(df_sst_lef_dek)

Moyenne pour la période 1961-1990 : 300.01800673876244
Moyenne pour la période 1961-1990 : 300.01800673876244


In [77]:
df_sst_lef_dek.head()

,Year,Month,Decade,anom_lef_dek
0,1961,1,1,0.365944
80,1961,1,2,0.232241
160,1961,1,3,0.668866
248,1961,2,1,0.866864
328,1961,2,2,1.098664


#### I.2 - CALCUL MENSUEL

In [78]:
df_sst_lef_mois=df_merged_sst_lef.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_lef_mois['Moy_lef_Mois'] = df_sst_lef_mois.groupby(['Year', 'Month'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_lef_mois.drop_duplicates(subset=['Year', 'Month', 'Moy_lef_Mois'], inplace=True)
# Affichez le DataFrame résultant
df_sst_lef_mois.head()

,date,sst,Year,Month,Day,Decade,Moy_lef_Mois
0,1961-01-01,300.468854,1961,1,1,1,300.448309
248,1961-02-01,300.968978,1961,2,1,1,301.100340
472,1961-03-01,301.283281,1961,3,1,1,301.556624
720,1961-04-01,301.666668,1961,4,1,1,301.702403
960,1961-05-01,301.620156,1961,5,1,1,301.258271


##### Calcul anomalies mensuelles

In [79]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_lef_mois[(df_sst_lef_mois['Year'] >= ref_start) & (df_sst_lef_mois['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_lef_mois['anom_lef_mois']=df_sst_lef_mois['Moy_lef_Mois']-sst_mean
df_sst_lef_mois=df_sst_lef_mois[['Year','Month','anom_lef_mois']]

df_mer_mon.append(df_sst_lef_mois)

Moyenne pour la période 1961-1990 : 300.01581629732374
Moyenne pour la période 1961-1990 : 300.01581629732374


In [80]:
df_sst_lef_mois.head()

,Year,Month,anom_lef_mois
0,1961,1,0.432493
248,1961,2,1.084524
472,1961,3,1.540807
720,1961,4,1.686587
960,1961,5,1.242455


### VIII- PROCESSING NINO 3.4 DATA

##### Changement des données manquantes du fichiers decadaires netcdf

In [81]:
# Variables de configuration
folder_original = Config.SST_NINO_DEK
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.SST_NINO_VAL
change_missing_val(folder_original,folder_miss_check)

##### Calcul de la moyenne spatiale

In [82]:
# Dossier d'entrée contenant les fichiers SST
input_directory = Config.SST_NINO_VAL

# Dossier de sortie pour les fichiers résultants
output_directory = Config.SST_NINO_SP_MN_PATH

# Vérifier les chemins des dossiers
print(f"Dossier d'entrée : {input_directory}")
print(f"Dossier de sortie : {output_directory}")

# Si le dossier de sortie existe, supprimez-le
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

# Recréez le dossier de sortie
os.makedirs(output_directory)

# Boucle sur les fichiers dans le dossier d'entrée
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        # Chemin complet vers le fichier d'entrée
        input_file = os.path.join(input_directory, filename)
        
        # Déterminez le nom du fichier de sortie en ajoutant "_fldmean" au nom d'origine
        output_filename = os.path.splitext(filename)[0] + '_fldmean.nc'
        output_file = os.path.join(output_directory, output_filename)
        
        # Ouvrir le fichier NetCDF avec xarray
        ds = xr.open_dataset(input_file)
        
        # Calculer la moyenne spatiale
        ds_mean = ds.mean(dim=['latitude', 'longitude'])
        
        # Sauvegarder le résultat dans un nouveau fichier NetCDF
        ds_mean.to_netcdf(output_file)
        
        #print(f"Fichier généré : {output_file}")

print("Calcul de la moyenne spatiale (fldmean) terminé.")


Dossier d'entrée : ..\ASSETS\DATA\RAW\ERA5\SST_NINO_VAL
Dossier de sortie : ..\ASSETS\DATA\RAW\ERA5\SST_NINO_FLDMEAN
Dossier d'entrée : ..\ASSETS\DATA\RAW\ERA5\SST_NINO_VAL
Dossier de sortie : ..\ASSETS\DATA\RAW\ERA5\SST_NINO_FLDMEAN


Calcul de la moyenne spatiale (fldmean) terminé.
Calcul de la moyenne spatiale (fldmean) terminé.


##### Concatenation des fichiers après moyenne spatiale

In [83]:
dir_out_era5 = Config.ERA5_PATH
output_merged_file = os.path.join(dir_out_era5, Config.SST_NINO_MERGE)
to_concat_directory = Config.SST_NINO_SP_MN_PATH

# Liste de fichiers de sortie à concaténer
output_files_to_concatenate = [os.path.join(to_concat_directory, filename) for filename in os.listdir(to_concat_directory) if filename.endswith('_fldmean.nc')]

datasets = [xr.open_dataset(f) for f in output_files_to_concatenate]

# Concaténez les datasets le long de la dimension temporelle
combined = xr.concat(datasets, dim='time')

# Construisez le chemin complet pour le fichier de sortie
output_merged_file= os.path.join(dir_out_era5, Config.SST_NINO_MERGE)

# Enregistrez le dataset combiné dans un fichier
combined.to_netcdf(output_merged_file)

print(f"Le fichier combiné a été enregistré à {output_merged_file}")


Le fichier combiné a été enregistré à ..\ASSETS\DATA\RAW\ERA5\sst_nino_merged.nc
Le fichier combiné a été enregistré à ..\ASSETS\DATA\RAW\ERA5\sst_nino_merged.nc


##### Transformation du fichier netcdf en dataframe

In [84]:
# Ouvrir le fichier NetCDF
data = netCDF4.Dataset(output_merged_file)
# Extraire les variables time et sst
time_values = data.variables['time'][:]
sst_values = data.variables['sst'][:]

# Redimensionner les tableaux pour les rendre 1D
time_values = time_values.flatten()
sst_values = sst_values.flatten()
# Extraire les attributs de la variable "time"
time_var = data.variables['time']
units = time_var.units  # Obtenez l'unité de temps
# Fermer le fichier NetCDF
data.close()

# Convertir les valeurs de "time" en dates en utilisant cftime
dates = cftime.num2date(time_values, units, calendar='standard')

# Créer un DataFrame Pandas avec les dates converties
df_merged_sst_nino = pd.DataFrame({'date': dates, 'sst': sst_values})

In [85]:
# Utilisez apply pour convertir chaque valeur en datetime
df_merged_sst_nino['date'] = df_merged_sst_nino['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if isinstance(x, cftime._cftime.DatetimeGregorian) else x)
df_merged_sst_nino['date'] = pd.to_datetime(df_merged_sst_nino['date'])

In [86]:
df_merged_sst_nino['Year'] = df_merged_sst_nino['date'].dt.year
# Créez la colonne 'Mois' en extrayant le mois
df_merged_sst_nino['Month'] = df_merged_sst_nino['date'].dt.month
# Créez la colonne 'Jour' en extrayant le jour
df_merged_sst_nino['Day'] = df_merged_sst_nino['date'].dt.day
df_merged_sst_nino['Decade']=pd.cut(df_merged_sst_nino['Day'],bins=[0,10,20,32], labels=[1,2,3])

#### VIII.1 - CALCUL DECADAIRE

In [87]:
df_sst_nino_dek=df_merged_sst_nino.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_nino_dek['Moy_nino_Dec'] = df_sst_nino_dek.groupby(['Year', 'Month','Decade'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_nino_dek.drop_duplicates(subset=['Year', 'Month','Decade','Moy_nino_Dec'], inplace=True, keep='last')
# Affichez le DataFrame résultant
#df_sst_nino_dek.head()

##### Calcul anomalies decadaires

In [88]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_nino_dek[(df_sst_nino_dek['Year'] >= ref_start) & (df_sst_nino_dek['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_nino_dek['anom_nino_dek']=df_sst_nino_dek['Moy_nino_Dec']-sst_mean
df_sst_nino_dek=df_sst_nino_dek[['Year','Month','Decade','anom_nino_dek']]
df_mer_dek.append(df_sst_nino_dek)

Moyenne pour la période 1961-1990 : 300.75231720814514
Moyenne pour la période 1961-1990 : 300.75231720814514


#### VIII.2 - CALCUL MENSUEL

In [89]:
df_sst_nino_mois=df_merged_sst_nino.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_nino_mois['Moy_nino_Mois'] = df_sst_nino_mois.groupby(['Year', 'Month'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_nino_mois.drop_duplicates(subset=['Year', 'Month','Moy_nino_Mois'], inplace=True, keep='last')
# Affichez le DataFrame résultant
df_sst_nino_mois.head()

,date,sst,Year,Month,Day,Decade,Moy_nino_Mois
247,1961-01-31 21:00:00,301.076386,1961,1,31,3,300.850966
471,1961-02-28 21:00:00,301.244899,1961,2,28,3,301.184179
719,1961-03-31 21:00:00,301.603948,1961,3,31,3,301.383963
959,1961-04-30 21:00:00,301.676068,1961,4,30,3,301.779199
1207,1961-05-31 21:00:00,301.377111,1961,5,31,3,301.479861


In [90]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_nino_mois[(df_sst_nino_mois['Year'] >= ref_start) & (df_sst_nino_mois['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_nino_mois['anom_nino_mois']=df_sst_nino_mois['Moy_nino_Mois']-sst_mean
df_sst_nino_mois=df_sst_nino_mois[['Year','Month','anom_nino_mois']]
df_mer_mon.append(df_sst_nino_mois)

Moyenne pour la période 1961-1990 : 300.7504925560242
Moyenne pour la période 1961-1990 : 300.7504925560242


### IX- Récapitulation et sauvegarde des dataframes

##### Fusion des dataframes des paramètres locaux (mesurés sur le Burkina)

##### Decadaires

In [91]:
# Fusion des dataframes de la liste df_terre_dek
df_terre_merg_dek = reduce(lambda left, right: pd.merge(left, right, on=['Station', 'Year', 'Month', 'Decade']), df_terre_dek)

# Afficher le dataframe fusionné
print(df_terre_merg_dek)
#df_terre_merg_dek.to_csv('./test_fusion.xlsx')

              Station  Year  Month Decade  v_wind_925  u_wind_850  u_wind_700  u_wind_200  eau_precipitable  t_point_rosee  h_vol_sol_wat
0      Bobo_Dioulasso  1961      1      1   -5.685185   -4.901044   -6.507310   24.359285      5.872143e-09     274.505384       0.177588
1      Bobo_Dioulasso  1961      1      2   -5.627637   -1.549461   -1.188504   28.219953      1.950160e-08     274.479259       0.178358
2      Bobo_Dioulasso  1961      1      3   -7.595885   -0.394695    2.207007   32.303318     -7.126785e-07     273.833101       0.178193
3      Bobo_Dioulasso  1961      2      1   -5.607470   -3.607335    3.229098   12.524779     -3.006926e-08     269.842157       0.172408
4      Bobo_Dioulasso  1961      2      2   -5.117176   -3.535849    3.110321   19.788904     -4.638627e-08     269.390330       0.173361
...               ...   ...    ...    ...         ...         ...         ...         ...               ...            ...            ...
22675              Po  2023     11

##### Mensuelles

In [92]:
# Fusion des dataframes de la liste df_terre_dek
df_terre_merg_mon = reduce(lambda left, right: pd.merge(left, right, on=['Station', 'Year', 'Month']), df_terre_mon)

# Afficher le dataframe fusionné
print(df_terre_merg_mon)
#df_terre_merg_mon.to_csv('./test_merg_terre_mon.xlsx')

             Station  Year  Month  v_wind_925  u_wind_850  u_wind_700  u_wind_200  eau_precipitable  t_point_rosee  h_vol_sol_wat
0     Bobo_Dioulasso  1961      1   -6.344612   -2.220861   -1.699389   28.423512     -2.447008e-07     274.258404       0.178051
1     Bobo_Dioulasso  1961      2   -4.787571   -4.507234    0.534085   17.062947     -4.154257e-08     270.738747       0.173297
2     Bobo_Dioulasso  1961      3   -2.121944   -3.204388   -0.900405   22.134290      1.196773e-07     275.947227       0.170007
3     Bobo_Dioulasso  1961      4    4.142627   -1.348264   -4.686925   13.424217      1.468938e-08     291.274081       0.204349
4     Bobo_Dioulasso  1961      5    3.927357   -0.323172   -8.638417    5.019922     -6.044959e-08     294.604530       0.262240
...              ...   ...    ...         ...         ...         ...         ...               ...            ...            ...
7555              Po  2023      8    2.812753    0.693429   -6.635411  -11.575755     -4.3

##### Fusion des dataframes des anomalies des températures de mers

##### Decadaire

In [93]:
# Fusion des dataframes de la liste df_terre_dek
df_mer_merg_dek = reduce(lambda left, right: pd.merge(left, right, on=['Year', 'Month', 'Decade']), df_mer_dek)
# Afficher le dataframe fusionné
print(df_mer_merg_dek)

      Year  Month Decade  anom_lef_dek  anom_nino_dek
0     1961      1      1      0.365944      -0.024139
1     1961      1      2      0.232241       0.064786
2     1961      1      3      0.668866       0.241058
3     1961      2      1      0.866864       0.408497
4     1961      2      2      1.098664       0.390542
...    ...    ...    ...           ...            ...
2263  2023     11      2      0.852772       0.962611
2264  2023     11      3      1.147468       1.141620
2265  2023     12      1      1.421022       1.197523
2266  2023     12      2      1.426720       1.206156
2267  2023     12      3      1.600289       1.281342

[2268 rows x 5 columns]
      Year  Month Decade  anom_lef_dek  anom_nino_dek
0     1961      1      1      0.365944      -0.024139
1     1961      1      2      0.232241       0.064786
2     1961      1      3      0.668866       0.241058
3     1961      2      1      0.866864       0.408497
4     1961      2      2      1.098664       0.390542
...

##### Mensuelle

In [94]:
# Fusion des dataframes de la liste df_terre_dek
df_mer_merg_mon = reduce(lambda left, right: pd.merge(left, right, on=['Year', 'Month']), df_mer_mon)
# Afficher le dataframe fusionné
print(df_mer_merg_mon)

     Year  Month  anom_lef_mois  anom_nino_mois
0    1961      1       0.432493        0.100473
1    1961      2       1.084524        0.433687
2    1961      3       1.540807        0.633470
3    1961      4       1.686587        1.028707
4    1961      5       1.242455        0.729368
..    ...    ...            ...             ...
751  2023      8      -1.068338        0.360447
752  2023      9      -0.501238        0.421027
753  2023     10       0.273432        0.717276
754  2023     11       0.899425        1.010427
755  2023     12       1.488661        1.231875

[756 rows x 4 columns]
     Year  Month  anom_lef_mois  anom_nino_mois
0    1961      1       0.432493        0.100473
1    1961      2       1.084524        0.433687
2    1961      3       1.540807        0.633470
3    1961      4       1.686587        1.028707
4    1961      5       1.242455        0.729368
..    ...    ...            ...             ...
751  2023      8      -1.068338        0.360447
752  2023      9

##### IX-1 Fusion des dataframes locaux et mer

##### Decadaires

In [95]:
# Fusionner les DataFrames en utilisant une fusion externe (outer) sans spécifier les colonnes de fusion
df_dek = df_terre_merg_dek.merge(df_mer_merg_dek, how='outer')
df_dek.to_csv(os.path.join(Config.FILES_TRAITED_PATH,Config.ERA_FUSION_DEK),index=False)

In [96]:
df_dek.head()

,Station,Year,Month,Decade,v_wind_925,u_wind_850,u_wind_700,u_wind_200,eau_precipitable,t_point_rosee,h_vol_sol_wat,anom_lef_dek,anom_nino_dek
0,Bobo_Dioulasso,1961,1,1,-5.685185,-4.901044,-6.507310,24.359285,5.872143e-09,274.505384,0.177588,0.365944,-0.024139
1,Bogande,1961,1,1,-4.457056,-5.318282,-3.077982,26.031411,7.469295e-08,268.932499,0.080787,0.365944,-0.024139
2,Boromo,1961,1,1,-5.117354,-4.519135,-5.647762,24.738232,2.742668e-08,274.025898,0.168335,0.365944,-0.024139
3,Dedougou,1961,1,1,-4.927709,-4.962412,-4.930288,26.618781,4.440598e-08,273.528833,0.123838,0.365944,-0.024139
4,Dori,1961,1,1,-3.655243,-6.004024,-1.298756,28.527198,-7.225495e-08,267.173088,0.181110,0.365944,-0.024139


##### Mensuelles

In [97]:
# Fusionner les DataFrames en utilisant une fusion externe (outer) sans spécifier les colonnes de fusion
df_mon = df_terre_merg_mon.merge(df_mer_merg_mon, how='outer')
df_mon.to_csv(os.path.join(Config.FILES_TRAITED_PATH,Config.ERA_FUSION_MON),index=False)

In [98]:
df_mon.head()

,Station,Year,Month,v_wind_925,u_wind_850,u_wind_700,u_wind_200,eau_precipitable,t_point_rosee,h_vol_sol_wat,anom_lef_mois,anom_nino_mois
0,Bobo_Dioulasso,1961,1,-6.344612,-2.220861,-1.699389,28.423512,-2.447008e-07,274.258404,0.178051,0.432493,0.100473
1,Bogande,1961,1,-5.767704,-2.073078,1.304748,30.424754,-6.035625e-08,269.085430,0.081025,0.432493,0.100473
2,Boromo,1961,1,-6.010698,-1.756913,-0.934060,29.054192,-9.627294e-08,273.843337,0.166356,0.432493,0.100473
3,Dedougou,1961,1,-5.853977,-2.094841,-0.325521,30.879742,-2.844164e-07,273.860768,0.122703,0.432493,0.100473
4,Dori,1961,1,-5.160844,-2.311904,2.618656,32.211017,3.345370e-07,268.276131,0.183776,0.432493,0.100473


In [99]:
df_mon3=calculate_rolling_average(df_mon, 3)

In [100]:
df_mon3[df_mon3['Station']=='Boromo'].tail(10)

,Station,Year,Month,v_wind_925,u_wind_850,u_wind_700,u_wind_200,eau_precipitable,t_point_rosee,h_vol_sol_wat,anom_lef_mois,anom_nino_mois
7462,Boromo,2023,3,-2.209508,-3.249172,-3.263510,17.632480,-3.213181e-08,279.851036,0.161972,1.497105,0.639252
7472,Boromo,2023,4,-0.516677,-2.696688,-3.650549,17.697879,1.165849e-07,283.545306,0.160328,1.906551,1.158635
7482,Boromo,2023,5,1.762595,-0.910161,-5.262005,13.656965,1.615834e-07,289.543756,0.169550,2.036728,1.509665
7492,Boromo,2023,6,3.254770,0.236576,-7.283803,4.404364,-3.047069e-08,293.699360,0.208712,1.534842,1.482042
7502,Boromo,2023,7,3.821783,0.415875,-9.602079,-3.764160,-2.599312e-07,296.207386,0.252774,0.544180,1.089830
7512,Boromo,2023,8,3.373881,0.202947,-8.932227,-9.025354,-1.424013e-07,296.782353,0.325670,-0.435900,0.675410
7522,Boromo,2023,9,2.676608,-0.916574,-9.606976,-9.296333,1.300724e-07,297.004245,0.360692,-0.751432,0.449519
7532,Boromo,2023,10,2.458865,-2.051772,-9.726823,-4.194618,4.283979e-07,296.595945,0.339118,-0.432048,0.499583
7542,Boromo,2023,11,1.759161,-3.046977,-8.955419,2.135772,3.052995e-07,294.261916,0.259618,0.223873,0.716243
7552,Boromo,2023,12,-0.185406,-3.430873,-6.124835,9.315534,2.151986e-07,288.594879,0.190183,0.887173,0.986526
